In [1]:
%conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
absl-py                   0.11.0                   pypi_0    pypi
alabaster                 0.7.12                   pypi_0    pypi
apex                      0.1                      pypi_0    pypi
argon2-cffi               20.1.0                   pypi_0    pypi
ascii-graph               1.5.1                    pypi_0    pypi
astunparse                1.6.3                    pypi_0    pypi
attrs                     19.3.0                     py_0  
audioread                 2.1.8                    pypi_0    pypi
babel                     2.8.0                    pypi_0    pypi
backcall                  0.2.0                      py_0  
beautifulsoup4            4.9.1                    py36_0  
blas                      1.0                    openblas  
bleach                    3.1.5                    pypi_0    p

In [ ]:
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, Model
from keras.layers import LSTM, Conv2D, Dense, Input, Reshape, Flatten, Concatenate, ZeroPadding2D, LocallyConnected2D, Add
from keras.callbacks import EarlyStopping
from keras import regularizers
import matplotlib.pyplot as plt

def rate_normalize(d):
    return (d - 0.8) / 0.4

def rate_denormalize(d):
    return (d * 0.4) + 0.8

def get_data(time_range, past=30, future=1):
    stocks = ['VFINX', '^SP500TR', 'VBMFX', 'NVDA', 'AMD', 'INTC', 'AAPL', 'GOOG', 'AMZN', 'CSCO', 'SNE', 'RTX', 'MMM', 'IBM', 'WMT', 'UNH', 'MRK', 'KO']
    attrs = ['Adj Close', 'Open', 'High', 'Low', 'Close']

    df = pdr.get_data_yahoo(symbols=stocks, start=time_range[0], end=time_range[1])[attrs]

    df = df.swaplevel(0, 1, axis=1)[stocks]
    x_raw = df.to_numpy().reshape((df.shape[0], len(stocks), len(attrs)))
    x_data = x_raw[1:] - x_raw[:-1]
    x_min = x_data.min(axis=0)
    x_max = x_data.max(axis=0)

    y_df = df['VFINX', 'Adj Close']
    y_raw = y_df.to_numpy()
    y_data = y_raw[1:] - y_raw[:-1]

    x = []
    y = []

    for i in range(y_df.shape[0] - past - future):
        x.append(x_data[i : (i + past)])
        y.append(y_data[(i + past) : (i + past + future)])

    return (y_df, np.array(x), np.array(y), x_data[-past:], x_min, x_max)


def sub_model(input_layer, layer_size, kernel_size, feature_dilation=[1, 3, 5, 9], time_dilation=1):
    tk, fk = kernel_size
    
    z = [Conv2D(layer_size, kernel_size=(1, 1), padding='valid')(input_layer)]
    
    for fd in feature_dilation:
        t = ZeroPadding2D(padding=(((tk - 1) * time_dilation, 0), (fd, fd)))(input_layer)
        t = Conv2D(layer_size, kernel_size=kernel_size, dilation_rate=(time_dilation, fd), padding='valid')(t)
        z.append(t)
    
    z = Concatenate()(z)
    
    return Conv2D(layer_size, kernel_size=(1, 1), padding='valid')(z)


def get_model(input_dim, past=30, future=1):
    c = 0.01

    x = Input(shape=input_dim)
    
    t0 = sub_model(input_layer=x, layer_size=60, kernel_size=(3, 3), feature_dilation=[1, 3, 5, 9], time_dilation=1)
    t1 = sub_model(input_layer=t0, layer_size=60, kernel_size=(3, 3), feature_dilation=[1, 3, 5, 9], time_dilation=3)
    
    r0 = Conv2D(60, kernel_size=(1, 1), padding='valid')(x)
    t1 = Add()([t1, r0])
    
    t2 = sub_model(input_layer=t1, layer_size=60, kernel_size=(3, 3), feature_dilation=[1, 3, 5, 9], time_dilation=5)
    t3 = sub_model(input_layer=t2, layer_size=60, kernel_size=(3, 3), feature_dilation=[1, 3, 5, 9], time_dilation=9)
    
    t3 = Add()([t3, t1])
    t4 = sub_model(input_layer=t3, layer_size=60, kernel_size=(3, 3), feature_dilation=[1, 3, 5, 9], time_dilation=17)
    t4 = LocallyConnected2D(1, kernel_size=(1, 1), padding='valid')(t4)
    m = Flatten()(t4)
    
    o = Dense(future)(m)
    model = Model(inputs=x, outputs=o)
    model.compile(loss='mse', optimizer='adam')
#     model.summary()

    return model,o

# def classify(reg_thi):  # reg_thi 是前三十筆回歸值

In [49]:
past = 150
future = 5
validation_sz = 200
recent_sz = 900

(data, X, Y, pX, dmin, dmax) = get_data((datetime(2006, 1, 1), datetime(2020, 11, 20)), past, future)
dim = pX.shape
last_time = data.index[-1]

model_g_m = get_model(input_dim=dim, past=past, future=future)[0]
model_g_o = get_model(input_dim=dim, past=past, future=future)[1]

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 18, 5)] 0                                            
__________________________________________________________________________________________________
zero_padding2d_60 (ZeroPadding2 (None, 152, 20, 5)   0           input_4[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_61 (ZeroPadding2 (None, 152, 24, 5)   0           input_4[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_62 (ZeroPadding2 (None, 152, 28, 5)   0           input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
def classify(reg_o):
    x=reg_o
    output = tf.nn.softmax(x)
    

In [ ]:
def predict_plot(model, future, data, x, y, img_file):
    predict_diff = model.predict(x).sum(axis = 1)
    predict_data = data.to_numpy()[:-future] + predict_diff
    real_data = data.to_numpy()[:-future] + y.sum(axis = 1)
    df = pd.DataFrame({'real': data[future:], 'real_computed': real_data, 'predict': predict_data})
    plt.figure()
    df.plot(figsize=(20, 8))
    plt.show()

past = 150
future = 5
validation_sz = 200
recent_sz = 900

(data, X, Y, pX, dmin, dmax) = get_data((datetime(2006, 1, 1), datetime(2020, 11, 20)), past, future)
dim = pX.shape
last_time = data.index[-1]

model = get_model(input_dim=dim, past=past, future=future)[0]

try:
    model.load_weights('stock-predict-weights')
    validation_sz = 14
    recent_sz = 120
    model.fit(X, Y, epochs=10, batch_size=10)
except:
    model.fit(X[:-validation_sz], Y[:-validation_sz], epochs=100, batch_size=10, validation_data=(X[-validation_sz:], Y[-validation_sz:]))


predict_diff = model.predict(pX.reshape((1,) + dim)).sum()

print('Predict at ', last_time)
print('Predict diff ', predict_diff)
print('Predict value ', data[-1] + predict_diff)

model.save_weights('stock-predict-weights')

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 18, 5)] 0                                            
__________________________________________________________________________________________________
zero_padding2d_100 (ZeroPadding (None, 152, 20, 5)   0           input_6[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_101 (ZeroPadding (None, 152, 24, 5)   0           input_6[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_102 (ZeroPadding (None, 152, 28, 5)   0           input_6[0][0]                    
____________________________________________________________________________________________

In [3]:
predict_plot(model, future, data[-recent_sz : -validation_sz], X[-recent_sz + 1: -validation_sz], Y[-recent_sz + 1: -validation_sz], 'train.png')
predict_plot(model, future, data[-validation_sz -1:], X[-validation_sz:], Y[-validation_sz:], 'test.png')

NameError: name 'predict_plot' is not defined